In [ ]:
import re
import pandas as pd

import torch
from konlpy.tag import Okt
from pykospacing import Spacing
from transformers import BertTokenizer, BertModel

In [ ]:
spacing = Spacing()

def preprocessing(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = spacing(text)
    return text

def get_stopwords():                                                                                                        # 불용어 리스트 가져오기
    stopwords = set()
    with open('../../../data/stopwords-ko.txt', 'r', encoding='utf-8') as file:
        for line in file:
            stopwords.add(line.strip())
    return stopwords

def morph_and_remove_stopwords(text, stopwords):                                                                            # 형태소 분석 및 불용어 제거 함수
    text = preprocessing(text)
    okt = Okt()                                                                                                             # 형태소 분석기 초기화
    morphs = okt.morphs(text)
    meaningful_words = [word for word in morphs if word not in stopwords]                                                   # 불용어 제거
    return ' '.join(meaningful_words)

In [ ]:
stopwords = get_stopwords()                                                                                                 # 불용어 리스트 가져오기

df = pd.read_csv(r'..\..\..\data\combined_questions.csv', encoding='cp949')                                                 #분석할 파일 불러오기.

df['processed_feature'] = df['feature'].apply(lambda x: morph_and_remove_stopwords(x, stopwords))                           # 데이터프레임의 'feature' 열에 대해 형태소 분석 및 불용어 제거 적용

tokenizer = BertTokenizer.from_pretrained('monologg/kobert')                                                                # KoBERT 모델과 토크나이저 로드
model = BertModel.from_pretrained('monologg/kobert')

In [ ]:
def get_kobert_embedding(text, tokenizer, model):                                                                           # KoBERT 임베딩 추출 함수
    if not text.strip():                                                                                                    # 빈 문자열 확인
        return [0] * model.config.hidden_size
    
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)                            # 텍스트를 토큰화하고 텐서로 변환
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    token_type_ids = inputs.get('token_type_ids')

    with torch.no_grad():                                                                                                   # 모델을 사용하여 입력 텐서를 처리
        outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

    cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()                                                    # 첫 번째 토큰 [CLS]의 임베딩을 사용
    return cls_embedding

df['feature_vector'] = df['processed_feature'].apply(lambda x: get_kobert_embedding(x, tokenizer, model))                   # KoBERT 벡터를 데이터프레임에 추가